In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Google Analytics 4和BigQuery ML进行游戏开发者的流失预测

在Colab中运行
在GitHub上查看
在Vertex AI Workbench中打开

## 目录

* [概述](#section-1)
* [目标](#section-2)
* [数据集](#section-3)
* [成本](#section-4)
* [创建一个BigQuery数据集](#section-5)
* [探索数据](#section-6)
* [准备训练数据](#section-7)
    * [为每个用户标识标签](#section-7-subsection-1)
    * [提取每个用户的人口统计数据](#section-7-subsection-2)
    * [提取每个用户的行为数据](#section-7-subsection-3)
    * [将标签、人口统计和行为数据合并为训练数据](#section-7-subsection-4)
* [使用BigQuery ML训练倾向模型](#section-8)
* [模型评估](#section-9)
    * [混淆矩阵：预测值与实际值](#section-9-subsection-1)
    * [ROC曲线](#section-9-subsection-2)
* [模型预测](#section-10)  
* [将预测表导出到云存储](#section-11)
* [清理工作](#section-12)

## 概述
<a name="section-1"></a>

本教程向您展示如何在BigQuery ML中训练、评估一个倾向模型，以预测用户在移动游戏中的留存情况，基于来自Google Analytics 4的应用测量数据。

了解更多关于[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction) 和了解更多关于[BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml#machine_learning_directly_in)。

### 目标
<a name="section-2"></a>

在本教程中，您将学习如何在BigQuery ML中训练、评估倾向模型。

本教程使用以下 Google Cloud ML 服务和资源：
* BigQuery。

执行的步骤包括：

* 探索在BigQuery上导出的 Google Analytics 4 数据。
* 使用人口统计数据、行为数据和标签（流失/非流失）准备训练数据。
* 使用BigQuery ML训练一个XGBoost模型。
* 使用BigQuery ML评估模型。
* 使用BigQuery ML对哪些用户会流失进行预测。

### 数据集
<a name="section-3"></a>

这个笔记本使用了[这个公开的BigQuery数据集](https://console.cloud.google.com/bigquery?p=firebase-public-project&d=analytics_153293282&t=events_20181003&page=table)，其中包含了来自一个名叫Flood It!的真实移动游戏应用的原始事件数据（[Android app](https://play.google.com/store/apps/details?id=com.labpixies.flood), [iOS app](https://itunes.apple.com/us/app/flood-it!/id476943146?mt=8)）。[数据架构](https://support.google.com/analytics/answer/7029846)源自于Google Analytics for Firebase，但与[Google Analytics 4](https://support.google.com/analytics/answer/9358801)具有相同的架构；本笔记本中的技术可以应用于Google Analytics for Firebase或Google Analytics 4的数据。

Google Analytics 4（GA4）采用了一个[基于事件的](https://support.google.com/analytics/answer/9322688)测量模型。事件提供了有关应用程序或网站上发生的情况的见解，例如用户操作、系统事件或错误。数据集中的每一行都是一个事件，具有与该事件相关的各种特征以嵌套格式存储在行中。虽然Google Analytics默认已经记录了许多类型的事件，但开发人员也可以自定义欲记录的事件类型。

请注意，由于不能简单地使用原始事件数据来训练机器学习模型，本笔记本向您展示了将原始数据预处理成适合用于分类模型训练数据的重要步骤。

### 成本
<a name="section-4"></a>

本教程使用Google Cloud的以下可计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)， [BigQuery的定价](https://cloud.google.com/bigquery/pricing) 和[Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 安装其他软件包

In [ ]:
! pip3 install --quiet --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]'

仅限Colab使用：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 开始之前

#### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下操作：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[找到项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench** 
- 如果已经通过验证，请不用进行任何操作。

**2. 本地 JupyterLab 实例**，请取消注释并运行。

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
- 查看所有身份验证选项，请参阅：[Google Cloud Platform Jupyter Notebook 认证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_NAME = "your-bucket-name-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

UUID

如果您正在进行实时教程会话，则可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 导入库并定义常量

In [ ]:
import os

import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.cloud.bigquery import Client

初始化BigQuery客户端

In [ ]:
client = Client(project=PROJECT_ID)

创建一个BigQuery数据集
<a name="section-5"></a>

如果您正在使用***Vertex AI Workbench托管笔记本实例***，每个以“#@bigquery”开头的单元格将是一个SQL查询。如果您正在使用Vertex AI Workbench用户管理的笔记本实例或Colab，它将是一个Markdown单元格。

在这个笔记本中，您可以在您的项目中创建一个数据集。要创建数据集，请运行以下单元格：

# @bigquery
-- 在BigQuery中创建一个数据集

CREATE SCHEMA bqmlga4
OPTIONS(
  location="us"
  )

（**可选**）如果您正在使用Vertex AI Workbench托管的笔记本实例，则一旦从BigQuery中显示结果在上述单元格中，点击**查询并加载为DataFrame**按钮并执行生成的代码存根以将数据获取到当前笔记本中作为数据帧。

*注意：默认情况下，数据加载到名为`df`的变量中，但如果需要在执行单元格之前更改此设置。*

In [ ]:
dataset_id = "bqmlga4" + "_" + UUID

In [ ]:
query = """
CREATE SCHEMA `{PROJECT_ID}.{dataset_id}`
OPTIONS(
  location="us"
  )
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id
)
query_job = client.query(query)

print(query_job.result())

##探索数据
<a name="section-6"></a>

样本数据集包含原始事件数据，如下所示：

In [ ]:
query = """
SELECT 
    *
FROM
  `firebase-public-project.analytics_153293282.events_*`
    
LIMIT 5
"""
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

查看谷歌分析4中使用的整体模式可能会有所帮助。如前所述，谷歌分析4使用基于事件的测量模型，这个数据集中的每一行都是一个事件。[查看完整模式和每列的详细信息](https://support.google.com/analytics/answer/7029846)。如上所示，特定列是嵌套记录，包含详细信息：

* `app_info` 应用程序信息
* `device` 设备
* `ecommerce` 电子商务
* `event_params` 事件参数
* `geo` 地理位置
* `traffic_source` 流量来源
* `user_properties` 用户属性
* `items` (GA4数据集中默认存在)
* `web_info` (GA4数据集中默认存在)

以下查询结果显示此数据集中有15K个用户和5.7M个事件。

#@bigquery
SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`

In [ ]:
query = """
SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`
"""
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

准备训练数据
<a name="section-7"></a>

您不能简单地使用原始事件数据来训练机器学习模型，因为它可能不是适合用作训练数据的正确形状和格式。因此，在这一部分，您将学习如何将原始数据预处理为适当的格式，以便用作分类模型的训练数据。

为了预测哪些用户将会“流失”或“回归”，用于分类的理想训练数据格式应该如下所示：

|用户ID|用户人口统计数据|用户行为数据|流失|
|-|-|-|-|
|用户1|（例如，国家，设备类型）|（例如，一段时间内他们做某事的次数）|1
|用户2|（例如，国家，设备类型）|（例如，一段时间内他们做某事的次数）|0
|用户3|（例如，国家，设备类型）|（例如，一段时间内他们做某事的次数）|1

训练数据的特点：
- 每行是一个独立的唯一用户ID
- 属性为**人口统计数据**
- 属性为**行为数据**
- 您希望训练模型预测的实际**标签**（例如，1 = 流失，0 = 返回）

您可以仅使用人口统计数据或行为数据训练模型，但将两者结合起来可能有助于创建更具预测性的模型。因此，在本节中，您将学习如何对原始数据进行预处理，以符合这种训练数据格式。

以下部分将指导您在将人口数据、行为数据和标签准备好之前，将它们全部合并为完整的训练数据集。步骤如下：

1. 为每个用户确定标签（流失或回归）
2. 提取每个用户的人口数据
3. 提取每个用户的行为数据
4. 将标签、人口数据和行为数据合并为训练数据。

#### 步骤1：识别每个用户的标签
<a name="section-7-subsection-1"></a>

原始数据集并没有一个简单地将用户标识为“流失”或“返回”的特征，因此，在本节中，您需要根据一些现有的列创建这个标签。

有许多种方式来定义用户流失，但在这份笔记本的目的中，您可以预测1天内的流失用户，即用户在首次使用App后的24小时内再也没有回来使用App。

换句话说，在用户首次与App互动后的24小时内：
- 如果用户之后没有显示任何事件数据，则被视为**流失用户**。
- 如果用户之后至少有一个事件数据点，则被视为**返回用户**。

您可能还想要删除那些可能永远不会回来的用户，这些用户在使用App仅几分钟后就离开，有时被称为“跳出”。例如，您可能只想在使用App至少10分钟的用户上构建模型（即没有跳出的用户）。

因此，您对本笔记本中**流失用户**的更新定义是：
> “任何在App上至少使用了10分钟，但在首次使用App后的24小时内再也没有使用App的用户”。

在SQL中，由于原始数据包含每位用户的所有事件，从他们的第一次接触（应用程序安装）到最后一次接触，您可以使用这些信息来创建两列：`churned`和`bounced`。

请看以下的SQL查询和结果:

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.returningusers AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3
    );

SELECT 
  * 
FROM 
  bqmlga4.returningusers 
LIMIT 100;

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.returningusers AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3
    );

SELECT 
  * 
FROM 
  {dataset_id}.returningusers 
LIMIT 100;
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#@bigquery 
SELECT 
  * 
FROM 
  bqmlga4.returningusers

In [ ]:
query = """
SELECT 
  * 
FROM 
  {dataset_id}.returningusers 
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

对于“churned”列，如果用户在首次互动后24小时内执行了操作，则“churned = 0”，否则如果他们最后一次操作仅在前24小时内，则“churned = 1”。

对于`bounced`列，如果用户的最后一次操作是在与应用的第一次接触之后的前十分钟内，则`bounced = 1`，否则`bounced = 0`。您可以使用此列稍后筛选训练数据，通过有条件地查询`bounced = 0`的用户。

您可能想知道这15,000用户中有多少人弹跳并返回？您可以运行以下查询来检查：

#@bigquery
SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    bqmlga4.returningusers
GROUP BY 1,2
ORDER BY bounced

#@bigquery
SELECT
    bounced,
    churned,
    COUNT (churned) as number_of_users
FROM
    bqmlga4.returningusers
GROUP BY 1,2
ORDER BY bounced

In [ ]:
query = """
SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    {dataset_id}.returningusers
GROUP BY 1,2
ORDER BY bounced
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

对于训练数据，您只使用“反弹= 0”的数据。根据15,000名用户，您可以看到有5,557名（约41％）用户在首次与应用程序接触的前十分钟内反弹，但在剩下的8,031名用户中，1,883名用户（约23％）在24小时后流失。

#@bigquery
SELECT
    SUM(churned=1)/COUNT(churned) as churn_rate
FROM
    bqmlga4.returningusers
WHERE bounced = 0

In [ ]:
query = """
SELECT
    COUNTIF(churned=1)/COUNT(churned) as churn_rate
FROM
    {dataset_id}.returningusers
WHERE bounced = 0
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

数据中有23%的流失率客户，这对于训练一个流失预测模型来说并不算糟糕。如果类别不平衡的情况较严重，可以考虑使用过采样或欠采样技术来平衡类别分布。

步骤2：提取每个用户的人口统计数据
<a name="section-7-subsection-2"></a>

这一部分专注于提取每个用户的人口统计信息。数据集中已经包含了关于用户的不同人口统计信息，包括`app_info`、`device`、`ecommerce`、`event_params`和`geo`。人口统计特征可以帮助模型预测在特定设备或国家的用户更有可能流失。

对于这份笔记，您可以从`geo.country`、`device.operating_system`和`device.language`开始。如果您正在使用自己的数据集并且具有可连接的第一方数据，这一部分是一个很好的机会，可以为每个用户添加任何谷歌分析4中不容易获得的额外属性。

请注意，用户的人口统计信息有时可能会发生变化（例如从一个国家搬到另一个国家）。为了简单起见，您只需使用谷歌分析4在用户上次参与应用程序时提供的人口统计信息，这可以通过`MAX(event_timestamp)`来表示。这样可以确保每个唯一用户都可以用一行表示。

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  bqmlga4.user_demographics
LIMIT 10

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  {dataset_id}.user_demographics
LIMIT 10
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#@bigquery
查询
  *
从
  bqmlga4.user_demographics

In [ ]:
query = """
SELECT
  *
FROM
  {dataset_id}.user_demographics
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

第三步：提取每个用户的行为数据
<a name="section-7-subsection-3"></a>

原始事件数据中的行为数据跨越多个事件 -- 因此是每个用户的多行。本节的目标是聚合和提取每个用户的行为数据，每个唯一用户会得到一行行为数据。

但你需要准备什么样的行为数据？由于本笔记本的最终目标是根据用户在安装应用程序后的前24小时内的活动来预测这个用户是否会在之后流失或返回，所以你需要在训练数据中使用前24小时的行为数据。之后，你也可以从`user_first_engagement`中提取一些额外的与时间相关的特征，比如首次参与的月份或日期。

Google Analytics会自动收集可用于分析行为的[具体事件](https://support.google.com/analytics/answer/6317485)。此外，还有游戏推荐的[事件](https://support.google.com/analytics/answer/6317494)。

作为第一步，你可以基于`event_name`探索数据集中存在的所有独特事件：

#@bigquery
SELECT
    event_name,
    COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 1
ORDER BY
   event_count DESC

#@bigquery
选择
    事件名称,
    COUNT(事件名称) as 事件数量
从
    `firebase-public-project.analytics_153293282.events_*`
按1分组
按
    事件数量 降序排序

In [ ]:
query = """
SELECT
    event_name,
    COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 1
ORDER BY
   event_count DESC
"""
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

对于本教程，要预测用户是否会流失或返回，您可以开始通过统计用户参与以下事件类型的次数：

* `user_engagement`
* `level_start_quickplay`
* `level_end_quickplay`
* `level_complete_quickplay`
* `level_reset_quickplay`
* `post_score`
* `spend_virtual_currency`
* `ad_reward`
* `challenge_a_friend`
* `completed_5_levels`
* `use_extra_steps`

在SQL中，您可以通过计算每个用户数据集中每个上述“event_names”发生的总次数来聚合行为数据。

如果您正在使用自己的数据集，您可能会有不同的事件类型可供聚合和提取。您的应用程序可能会将非常不同的“event_names”发送到Google Analytics中，因此请确保使用适合您场景的事件。

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.user_aggregate_behavior AS (
WITH
  events_first24hr AS (
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      bqmlga4.returningusers r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
  );

SELECT
  *
FROM
  bqmlga4.user_aggregate_behavior
LIMIT 10

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.user_aggregate_behavior AS (
WITH
  events_first24hr AS (
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      {dataset_id}.returningusers r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
  );

SELECT
  *
FROM
  {dataset_id}.user_aggregate_behavior
LIMIT 10

""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

步骤4：将标签、人口统计数据和行为数据组合在一起作为训练数据。

在这一部分，您现在可以将这三个中间视图（标签、人口统计数据和行为数据）合并为最终的训练数据集。在这里，您还可以指定 `bounced = 0`，以便将训练数据限制在在使用应用程序的前10分钟内没有“弹回”的用户身上。

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.train AS (
    
  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.month,
    ret.julianday,
    ret.dayofweek,
    ret.churned
  FROM
    bqmlga4.returningusers ret
  LEFT OUTER JOIN
    bqmlga4.user_demographics dem
  ON 
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    bqmlga4.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
  );

SELECT
  *
FROM
  bqmlga4.train
LIMIT 10

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.train AS (
    
  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.month,
    ret.julianday,
    ret.dayofweek,
    ret.churned
  FROM
    {dataset_id}.returningusers ret
  LEFT OUTER JOIN
    {dataset_id}.user_demographics dem
  ON 
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    {dataset_id}.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
  );

SELECT
  *
FROM
  {dataset_id}.train
LIMIT 10
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#@bigquery
选择
  *
从
  bqmlga4.train

In [ ]:
query = """
SELECT
  *
FROM
  {dataset_id}.train
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
df = query_job.to_dataframe()

In [ ]:
df

检查空值百分比。

In [ ]:
(
    100 * df[["operating_system", "language", "country"]].isna().sum() / df.shape[0]
).plot.bar(figsize=(15, 4))
plt.title("Null-percentage of the columns")
plt.show()

用BigQuery ML训练倾向性模型

在本部分中，使用您准备的训练数据，您现在可以使用BigQuery ML 在SQL中训练机器学习模型。

您在这里使用[XGBoost]（https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree）模型。在这个笔记本中，该模型预测用户在首次使用应用程序后24小时内是流失（1）还是返回（0）。

训练一个 XGBoost 模型

以下代码训练了一个XGBoost模型。这可能需要几分钟。

有关使用的默认超参数的更多信息，请参阅[使用XGBoost创建增强树模型的CREATE MODEL语句](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree)。

#@bigquery
CREATE OR REPLACE MODEL bqmlga4.churn_xgb

OPTIONS(
  MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
  DATA_SPLIT_METHOD='RANDOM',
  DATA_SPLIT_EVAL_FRACTION=0.2,
    
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  *
FROM
  bqmlga4.train

在BigQuery中创建或替换模型bqmlga4.churn_xgb，并设置选项（模型类型为“BOOSTED_TREE_CLASSIFIER”，数据分割方法为“RANDOM”，数据分割评估分数为0.2，输入标签列为“churned”），将所有数据从bqmlga4.train中选取。

In [ ]:
query = """
CREATE OR REPLACE MODEL {dataset_id}.churn_xgb

OPTIONS(
  MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
  DATA_SPLIT_METHOD='RANDOM',
  DATA_SPLIT_EVAL_FRACTION=0.2,
    
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  *
FROM
  {dataset_id}.train
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.result()

## 模型评估
<a name="section-9"></a>

要评估模型，您可以在训练完成的模型上运行[`ML.EVALUATE`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate)以检查一些指标。

这些指标基于在模型创建过程中自动拆分的测试样本数据（[请参阅创建模型文档获取更多信息](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method)）。

#@bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlga4.churn_xgb)

In [ ]:
query = """
SELECT
  *
FROM
  ML.EVALUATE(MODEL {dataset_id}.churn_xgb)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

`ML.EVALUATE`使用默认的分类阈值0.5生成`precision`、`recall`、`accuracy`和`f1_score`，这个阈值可以通过使用可选的[`THRESHOLD`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#eval_threshold)参数进行修改。

通常情况下，您可以使用`log_loss`和`roc_auc`指标来比较模型性能。

`log_loss`的范围在0到1.0之间，`log_loss`越接近于零，预测标签就越接近实际标签。

`roc_auc`的范围在0到1.0之间，`roc_auc`越接近1.0，模型在区分不同类别方面就越好。

有关这些指标的更多信息，您可以阅读关于[precision和recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall)、[accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy)、[f1-score](https://en.wikipedia.org/wiki/F-score)、[log_loss](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Logistic_loss)和[roc_auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)的定义。

混淆矩阵：预测值与实际值
<a name="section-9-subsection-1"></a>

除了模型评估指标之外，您可能还想使用混淆矩阵来检查模型预测标签与实际标签相比的准确度。

以实际标签表示行，以预测标签表示列，二元分类的ML.CONFUSION_MATRIX 的结果格式如下：

| | 预测为0 | 预测为1|
|-|-|-|
|实际为0| 真阴性(True Negatives) | 假阳性(False Positives)|
|实际为1| 假阴性(False Negatives) | 真阳性(True Positives)|

有关混淆矩阵的更多信息，请参阅[分类: 真正与假负以及正例与反例](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)。

#@bigquery
选择
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
从
  ML.CONFUSION_MATRIX(MODEL bqmlga4.churn_xgb)

In [ ]:
query = """
SELECT
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
FROM
  ML.CONFUSION_MATRIX(MODEL {dataset_id}.churn_xgb)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

ROC 曲线
<a name="section-9-subsection-2"></a>

#@bigquery
SELECT * FROM ML.ROC_CURVE(MODEL bqmlga4.churn_xgb) 

#@大型查询
选择* FROM ML.ROC_CURVE(MODEL bqmlga4.churn_xgb)

In [ ]:
query = """
SELECT * FROM ML.ROC_CURVE(MODEL {dataset_id}.churn_xgb)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

## 模型预测
<a name="section-10"></a>

您可以运行[`ML.PREDICT`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict)来预测流失的倾向。以下代码返回`ML.PREDICT`中的所有信息。

创建或替换视图bqmlga4.prediction_data AS(
(SELECT * FROM bqmlga4.train WHERE churned=1 LIMIT 10)
UNION ALL
(SELECT * FROM bqmlga4.train WHERE churned=0 LIMIT 20))

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.prediction_data AS(
(SELECT * FROM {dataset_id}.train where churned=1 limit 10)
union all
(SELECT * FROM {dataset_id}.train where churned=0 limit 20))
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.result()

#@bigquery
SELECT
  *
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bqmlga4.prediction_data))

In [ ]:
query = """
SELECT
  *
FROM
  ML.PREDICT(MODEL {dataset_id}.churn_xgb,
  (SELECT * FROM {dataset_id}.prediction_data)
            ) 
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

对于倾向性建模来说，最重要的输出是某种行为发生的概率。以下查询将返回用户在24小时后返回的概率。概率越高，越接近于1，用户更可能被预测为流失，概率越接近于0，用户更可能被预测为返回。

#@bigquery
SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned

FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bqmlga4.train)) 

中文：
#@bigquery
选择
  用户伪ID，
  流失，
  预测流失，
  预测流失概率[偏移（0）]。概率作为概率流失

从
  ML. 预测(模型 bqmlga4.churn_xgb,
  （选择*从bqmlga4.train）)

In [ ]:
query = """
SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
  
FROM
  ML.PREDICT(MODEL {dataset_id}.churn_xgb,
  (SELECT * FROM {dataset_id}.train))
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

将预测表导出至云存储
<a name="section-11"></a>

有几种将预测表导出到云存储的方法，这样您可以在单独的服务中使用这些信息。也许最简单的方法是使用SQL直接将数据导出到云存储。[了解更多关于EXPORT DATA语句的信息](https://cloud.google.com/bigquery/docs/reference/standard-sql/other-statements#export_data_statement)。

In [ ]:
query = """
CREATE OR REPLACE TABLE {dataset_id}.prediction_data_table AS (
SELECT 
  * 
FROM 
  {dataset_id}.prediction_data
)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.result()

In [ ]:
FILE_PATH = BUCKET_URI + "/" + "*.csv"

In [ ]:
query = """
EXPORT DATA OPTIONS (
uri= @FILE_PATH, 
  format=CSV,
  header=True, 
  overwrite=True 
    
) AS 
SELECT
  * from {dataset_id}.prediction_data_table
""".format(
    dataset_id=dataset_id
)

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("FILE_PATH", "STRING", FILE_PATH),
    ]
)
query_job = client.query(query, job_config=job_config)  # Make an API request.

In [ ]:
query_job.result()

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。以下代码将删除整个数据集。

In [ ]:
# Set dataset variable to the ID of the BigQuery dataset to fetch.
dataset = f"{PROJECT_ID}.{dataset_id}"

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    dataset, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(dataset_id))

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI